In [1]:
import os
import re
import ast
import json
import spacy
import pickle
import difflib
import requests
import datetime
import jsonlines
import numpy as np
import pandas as pd
from utils.shared_configs import LLAMA_MODEL_PATH, ZEHPYR_MODEL_PATH, get_sampling_params, initialize_llm
nlp = spacy.load("en_core_web_sm")

2025-05-08 00:21:15.013166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 00:21:15.024232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746663675.037918  715151 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746663675.042096  715151 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-08 00:21:15.056211: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
llm = initialize_llm(model_path=LLAMA_MODEL_PATH, tokenizer_path=LLAMA_MODEL_PATH)
sampling_params = get_sampling_params(max_tokens=200, temperature=0.6, top_p=0.9, stops=["</s>", "\n}"])

INFO 05-08 00:21:21 __init__.py:207] Automatically detected platform cuda.
INFO 05-08 00:21:33 config.py:549] This model supports multiple tasks: {'score', 'embed', 'classify', 'reward', 'generate'}. Defaulting to 'generate'.
WARNING 05-08 00:21:34 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 05-08 00:21:34 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 05-08 00:21:34 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f', speculative_config=None, tokenizer='/datasets/ai/llama3/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/5206a32e0bd3067aef1ce90f5528ade7d866253f', skip

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 05-08 00:21:38 model_runner.py:1115] Loading model weights took 14.9888 GB
INFO 05-08 00:21:39 worker.py:267] Memory profiling takes 0.73 seconds
INFO 05-08 00:21:39 worker.py:267] the current vLLM instance can use total_gpu_memory (44.34GiB) x gpu_memory_utilization (0.90) = 39.91GiB
INFO 05-08 00:21:39 worker.py:267] model weights take 14.99GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 23.67GiB.
INFO 05-08 00:21:39 executor_base.py:111] # cuda blocks: 12119, # CPU blocks: 2048
INFO 05-08 00:21:39 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 1.48x
INFO 05-08 00:21:41 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:19<00:00,  1.82it/s]

INFO 05-08 00:22:01 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.26 GiB
INFO 05-08 00:22:01 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 22.33 seconds


### Load WN-Salience

In [2]:
# wn_salience_train = "/project/pi_wenlongzhao_umass_edu/8/data/WN-Salience-2/SED_train.json"
# with open(wn_salience_train) as f:
#     sed_outputs = json.load(f)

In [50]:
# train_path = "/work/pi_wenlongzhao_umass_edu/8/riya/TrainValDataSplit/WN_salience_TrainSplit.pkl"
# val_path = "/work/pi_wenlongzhao_umass_edu/8/riya/TrainValDataSplit/WN_salience_ValSplit.pkl"

train_path = "/work/pi_wenlongzhao_umass_edu/8/696-detecting-salient-entities/data/WN_csv/WN_salience_TrainSplit.csv"
val_path = "/work/pi_wenlongzhao_umass_edu/8/696-detecting-salient-entities/data/WN_csv/WN_salience_ValSplit.csv"

In [4]:
# with open(val_path, "rb") as f:
#     wn_val = pickle.load(f)

/tmp/ipykernel_712181/2559814351.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  wn_val = pickle.load(f)


In [51]:
wn_val = pd.read_csv(val_path)

In [52]:
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15524 entries, 0 to 15523
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             15524 non-null  object
 1   date             15524 non-null  object
 2   title            15524 non-null  object
 3   entity title     15523 non-null  object
 4   entity salience  15524 non-null  int64 
 5   offsets          15524 non-null  object
 6   wiki_ID          15524 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 849.1+ KB


In [53]:
wn_val.head()

,text,date,title,entity title,entity salience,offsets,wiki_ID
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",4689264
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",32339793
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",5405
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",848348
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",27862


## Pointwise EL

### Preprocessing
1. Rename columns
2. Add surrounding context
3. Assign WikiID based on Wiki url

In [54]:
column_mapping = {
    'text': 'article_text',
    'title': 'article_title',
    'entity title': 'entity_title',
    'entity salience': 'entity_salience'
}
wn_val.rename(columns=column_mapping, inplace=True)

In [55]:
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15524 entries, 0 to 15523
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   article_text     15524 non-null  object
 1   date             15524 non-null  object
 2   article_title    15524 non-null  object
 3   entity_title     15523 non-null  object
 4   entity_salience  15524 non-null  int64 
 5   offsets          15524 non-null  object
 6   wiki_ID          15524 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 849.1+ KB


In [56]:
wn_val[wn_val['entity_title'].isna()==True]

,article_text,date,article_title,entity_title,entity_salience,offsets,wiki_ID
13909,On Sunday in tennis's French Open men's single...,2017-06-13,¡La décima! Nadal again a French CHAMP10N,NaN,0,"(277, 277)",-1


In [57]:
wn_val = wn_val.dropna(subset=['entity_title'])
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15523 entries, 0 to 15523
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   article_text     15523 non-null  object
 1   date             15523 non-null  object
 2   article_title    15523 non-null  object
 3   entity_title     15523 non-null  object
 4   entity_salience  15523 non-null  int64 
 5   offsets          15523 non-null  object
 6   wiki_ID          15523 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 970.2+ KB


In [60]:
wn_val = wn_val[wn_val['wiki_ID'] != -1]

In [61]:
wn_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15299 entries, 0 to 15523
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   article_text     15299 non-null  object
 1   date             15299 non-null  object
 2   article_title    15299 non-null  object
 3   entity_title     15299 non-null  object
 4   entity_salience  15299 non-null  int64 
 5   offsets          15299 non-null  object
 6   wiki_ID          15299 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 956.2+ KB


In [62]:
wn_val_subset = wn_val.head(250).copy()

In [63]:
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,wiki_ID
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",4689264
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",32339793
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",5405
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",848348
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",27862


In [73]:
def spacy_sent_tokenize(text):
    return [sent.text for sent in nlp(text).sents]

def extract_surrounding_context(article_text, offset, entity_title, n=1):
    """
    Return n sentences before and after the entity sentence, 
    with entity mention highlighted with ###mention###.
    """
    offset = ast.literal_eval(offset)
    start, end = offset    
    sents = list(nlp(article_text).sents)

    for i, sent in enumerate(sents):
        if sent.start_char <= start < sent.end_char:
            # Pick n sentences before and after
            begin = max(i - n, 0)
            end_ = min(i + n + 1, len(sents))
            context_sentences = [s.text for s in sents[begin:end_]]
            context = " ".join(context_sentences)
            entity_pattern = re.escape(entity_title)
            highlighted_context = re.sub(
                entity_pattern,
                f"###{entity_title}###",
                context,
                count=1  # Only highlight the first occurrence
            )
            return highlighted_context
    return ""

In [74]:
wn_val_subset['surrounding_context'] = wn_val_subset.apply(lambda row: extract_surrounding_context(row['article_text'], row['offsets'], row['entity_title'], n=2), axis=1)

In [75]:
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,wiki_ID,surrounding_context
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",4689264,"Homebush Bay, New South Wales —Earlier today, ..."
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",32339793,"Homebush Bay, New South Wales —Earlier today, ..."
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",5405,"Homebush Bay, New South Wales —Earlier today, ..."
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",848348,"Homebush Bay, New South Wales —Earlier today, ..."
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",27862,"Homebush Bay, New South Wales —Earlier today, ..."


In [15]:
# def get_wiki_id(url):
#     if not url:
#         return None
    
#     # title_match = re.search(r"wiki/(.+)$", url)
#     title_match = re.search(r"wiki/([^#?]+)", url)
#     if not title_match:
#         return None
    
#     title = title_match.group(1)
#     api_url = f"https://en.wikipedia.org/w/api.php?action=query&titles={title}&format=json"
    
#     try:
#         response = requests.get(api_url)
#         response_data = response.json()
#         pages = response_data.get("query", {}).get("pages", {})
        
#         if pages:
#             return next(iter(pages))  # Extract the first (and only) page ID
#     except Exception as e:
#         print(f"Error fetching Wiki ID for {url}: {e}")
    
#     return None

# wn_val_subset['gt_wiki_id'] = wn_val_subset['url'].apply(get_wiki_id)

In [77]:
wn_val_subset.to_csv('outputs/pointwise/wn_val_subset_5_7.csv', index=False)

In [64]:
wn_val_subset = pd.read_csv('outputs/pointwise/wn_val_subset_5_7.csv', dtype={'gt_wiki_id': 'Int64'})
wn_val_subset.head()

,article_text,date,article_title,entity_title,entity_salience,offsets,wiki_ID,surrounding_context
0,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Australian,1,"(50, 60)",4689264,"Homebush Bay, New South Wales —Earlier today, ..."
1,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Gliders,1,"(61, 68)",32339793,"Homebush Bay, New South Wales —Earlier today, ..."
2,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Chinese,1,"(78, 85)",5405,"Homebush Bay, New South Wales —Earlier today, ..."
3,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,wheelchair basketball,1,"(103, 124)",848348,"Homebush Bay, New South Wales —Earlier today, ..."
4,"Homebush Bay, New South Wales —Earlier today, ...",2012-07-20,Australian Gliders glide past China women's na...,Sydney,1,"(221, 227)",27862,"Homebush Bay, New South Wales —Earlier today, ..."


### Candidate retrieval

In [4]:
with open('/work/pi_wenlongzhao_umass_edu/8/aranade/696-detecting-salient-entities/prep_kb/filtered_kb_4_22.json') as f:
    filtered_kb = json.load(f)
    
alias_kb = {}
for entry in filtered_kb:
    key = entry['mentions'].lower()
    alias_kb.setdefault(key, []).append(entry)
    # cid = entry['wiki_id']
    # candidate_freq[cid] = candidate_freq.get(cid, 0) + 1

# max_freq, min_freq = max(candidate_freq.values()), min(candidate_freq.values())

len(alias_kb)

29037

In [5]:
def retrieve_candidates(mention, alias_kb):
    return alias_kb.get(mention.lower())

In [7]:
# import copy

# def retrieve_candidates(mention, cutoff=0.8, alpha=0.5, k=10):
#     mention_lower = mention.lower()
#     close_keys = difflib.get_close_matches(mention_lower, alias_kb.keys(), n=len(alias_kb), cutoff=cutoff)

#     # de‑dup while collecting
#     seen, uniq = set(), []
#     for key in close_keys:
#         for cand in alias_kb[key]:
#             cid = cand['cand_wiki_id']
#             # already collected
#             if cid in seen:
#                 continue
#             seen.add(cid)

#             new_cand = copy.deepcopy(cand)
#             # weighted score = α·fuzzy + (1‑α)·norm_freq
#             fuzzy = difflib.SequenceMatcher(None, mention_lower, cand['mention'].lower()).ratio()
#             freq  = candidate_freq[cid]
#             norm_freq = (freq - min_freq) / (max_freq - min_freq) if max_freq > min_freq else 1
#             new_cand['weighted_score'] = alpha * fuzzy + (1 - alpha) * norm_freq
#             uniq.append(new_cand)

#     return sorted(uniq, key=lambda c: c['weighted_score'], reverse=True)[:k]

### PROMPT TEMPLATE  (point‑wise)

In [11]:
def pointwise_el_prompt(entity, candidate):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for: 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. <|eot_id|><|start_header_id|>user<|end_header_id|>\n Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'. mention: {entity['entity_title']} context: {entity['surrounding_context']} candidate entity: {candidate['title']}.{candidate['text'][:250]} Output the JSON object in a single line without any line breaks, indentation, or extra spaces. Only output the JSON object. Do not output anything else. Here is the JSON structure to follow: {{"final_decision": "yes or no", "reasoning": "your explanation here"}} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""" 
    return prompt

In [54]:
# def pointwise_el_chat(entity, candidate):
#     return [
#         {
#             "role": "user", 
#             "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'. mention: {entity['entity_title']} context: {entity['surrounding_context']}" candidate entity: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be \"yes\" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the mention and the candidate entity are related. Output the JSON object in a single line without any line breaks, indentation, or extra spaces. Only output the JSON object. Do not output anything else. Here is the JSON structure to follow:"{{"final_decision": "yes or no","reasoning": "your explanation here"}}""")
#         }
#     ]

In [12]:
def pointwise_el_chat(entity, candidate):
    return [
        {
            "role": "user", 
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Now, I will provide you with a mention, its context, and a candidate entity. The mention in the context is highlighted with '###'. mention: {entity['entity_title']} context: {entity['surrounding_context']}" candidate entity: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be \"yes\" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the mention and the candidate entity are related. Output the JSON object in a single line without any line breaks, indentation, or extra spaces. Only output the JSON object. Do not output anything else. Here is the JSON structure to follow:"{{"final_decision": "yes or no","reasoning": "your explanation here"}}""")
        }
    ]

In [14]:
def pointwise_el_chat(entity, candidate):
    return [
        {
            "role": "user", 
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Your job is to decide whether a candidate should stay on the shortlist for an entity in context. 1. Output **"yes"** if the candidate is clearly the correct entity or you are uncertain / information is insufficient. Output **"no"** only if the candidate is clearly impossible. Now, I will provide you with an entity, its context, and a candidate with its description. The entity in the context is highlighted with '###'. entity: {entity['entity_title']} context: {entity['surrounding_context']}" candidate: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be \"yes\" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the entity and the candidate are related. Keep the answer to exactly one compact JSON object on a single line, no extra spaces: {{"final_decision":"yes|no","reasoning":""}}""")
        }
    ]

In [21]:
def construct_pointwise_prompt(entity, candidate):
    return [
        {
            "role": "user", 
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Your job is to decide whether a candidate should stay on the shortlist for an entity in context. 1. Output **"yes"** if the candidate is clearly the correct entity or you are uncertain or information is insufficient. 2. Output **"no"** only if the candidate is clearly impossible. Example 1: entity: Australian. context: "Earlier today, the ###Australian### Gliders beat the Chinese women's national wheelchair basketball team 57–45 at the Rollers & Gliders World Challenge." candidate: Australia — A country in the Southern Hemisphere that includes the mainland of the Australian continent, Tasmania, and numerous smaller islands. {{"final_decision":"yes","reasoning":"The mention 'Australian' modifies 'Gliders', a known Australian team. While the candidate is a country, it is plausible and commonly used in such adjectival form."}} Example 2: entity: Gliders. context: "Earlier today, the Australian ###Gliders### beat the Chinese women's national wheelchair basketball team."  candidate: Glider (aircraft) — A heavier-than-air aircraft that is supported in flight by the dynamic reaction of the air against its lifting surfaces.  {{"final_decision":"no","reasoning":"The candidate refers to an aircraft, which is not relevant here. In the context of a sports match, 'Gliders' refers to a sports team, not a flying machine."}} Now, I will provide you with an entity, its context, and a candidate with its description. The entity in the context is highlighted with '###'. entity: {entity['entity_title']} context: {entity['surrounding_context']}" candidate: {candidate['title']}. {candidate['text'][:300]}. Remember, the final answer must be "yes" when you believe the available information is insufficient or uncertain. Please analyze the information and determine if the entity and the candidate are related. Keep the answer to exactly one compact JSON object on a single line, no extra spaces: {{"final_decision":"yes|no","reasoning":""}}""")
        }
    ]

In [83]:
def construct_pointwise_prompt(entity, candidate):
    return [
        {
            "role": "user", 
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Your job is to decide whether a candidate should stay on the shortlist for an entity in context. Output **"yes"** if the candidate is a **plausible match** for the entity mention based on the context. Output **"no"** if the candidate is **clearly irrelevant or incompatible** with the context, even if the entity names look similar. Do **not** default to "yes" if you're unsure — use clues from the context, entity type, and modifiers to guide your decision. Example 1: entity: Australian. context: "Earlier today, the ###Australian### Gliders beat the Chinese women's national wheelchair basketball team 57–45 at the Rollers & Gliders World Challenge." candidate: Australia — A country in the Southern Hemisphere that includes the mainland of the Australian continent, Tasmania, and numerous smaller islands. {{"final_decision":"yes","reasoning":"The mention 'Australian' modifies 'Gliders', a known Australian team. While the candidate is a country, it is plausible and commonly used in such adjectival form."}} Example 2: entity: Gliders. context: "Earlier today, the Australian ###Gliders### beat the Chinese women's national wheelchair basketball team."  candidate: Glider (aircraft) — A heavier-than-air aircraft that is supported in flight by the dynamic reaction of the air against its lifting surfaces.  {{"final_decision":"no","reasoning":"The candidate refers to an aircraft, which is not relevant here. In the context of a sports match, 'Gliders' refers to a sports team, not a flying machine."}} Now, I will provide you with an entity, its context, and a candidate with its description. The entity in the context is highlighted with '###'. entity: {entity['entity_title']} context: {entity['surrounding_context']}" candidate: {candidate['title']}. {candidate['text'][:300]}. Please analyze the information and determine if the entity and the candidate are related. Keep the answer to exactly one compact JSON object on a single line, no extra spaces: {{"final_decision":"yes|no","reasoning":""}}""")
        }
    ]

### Sample run

In [66]:
et = wn_val_subset.iloc[0]
et['entity_title']

'Australian'

In [67]:
et['article_text']

"Homebush Bay, New South Wales —Earlier today, the Australian Gliders beat the Chinese women's national wheelchair basketball team 57–45 at the Rollers & Gliders World Challenge taking place at the Sport Centre at the Sydney Olympic Park.China won the game's opening tip off but did not score in their first time down the floor. Australia's Sarah Stewart took and missed Australia's first shot. China's 1.5 point player Li Yanhua eventually made the game's first basket. Cobi Crispin made Australia's first basket at 6:55 left in the first quarter, successfully turning her basket into a three point play by making her free throw as a result of a personal foul. Aggressive first quarter play resulted in four Chinese personal fouls with 5:21 left. The first quarter ended 13–9 in China's favour.The first half ended 24–20, with China leading. China's lead came with a high level of energy, the bench loudly supporting their players, few substitutions, lots of on the court talk, playing a half court 

In [68]:
et['surrounding_context']

"Homebush Bay, New South Wales —Earlier today, the ###Australian### Gliders beat the Chinese women's national wheelchair basketball team 57–45 at the Rollers & Gliders World Challenge taking place at the Sport Centre at the Sydney Olympic Park. China won the game's opening tip off but did not score in their first time down the floor. Australia's Sarah Stewart took and missed Australia's first shot."

In [69]:
candidates=retrieve_candidates(et['entity_title'], alias_kb)
len(candidates)

58

In [70]:
prompt1=pointwise_el_chat(et, candidates[0])
op1 = llm.chat(messages=prompt1, sampling_params=sampling_params)
op1[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it, est. speed input: 226.17 toks/s, output: 32.93 toks/s]


'{"final_decision": "no","reasoning": "The context mentions Homebush Bay, New South Wales, and the Australian Gliders, indicating that \'Australian\' refers to the team, not the country. The use of \'Australian\' as an adjective to describe the team suggests a nationality, rather than a geographic location."}'

In [71]:
decision(op1[0].outputs[0].text)

False

In [72]:
prompt2=pointwise_el_prompt(et, candidates[0])
op2 = llm.generate(prompts=prompt2, sampling_params=sampling_params)
op2[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it, est. speed input: 178.80 toks/s, output: 32.83 toks/s]


'\n\n{"final_decision": "yes", "reasoning": "The mention ###Australian### is in the context of a sports team name, and the candidate entity is the country of Australia, which is consistent with the context of the sports team name, especially considering the country\'s name is often used in team names, such as the Australian national wheelchair basketball team."}'

In [73]:
decision(op2[0].outputs[0].text)

True

In [74]:
prompt3=construct_pointwise_prompt(et, candidates[0])
op3 = llm.chat(messages=prompt3, sampling_params=sampling_params)
op3[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it, est. speed input: 539.76 toks/s, output: 31.03 toks/s]


'{"final_decision":"yes","reasoning":"The mention \'Australian\' modifies \'Gliders\', a known Australian team. The candidate is a country that the team is from, making it a plausible match."}'

In [75]:
decision(op3[0].outputs[0].text)

True

In [14]:
# def decision(text: str) -> bool:
#     try:
#         text = text.strip()

#         # If text already starts with { and ends with }, treat it directly
#         if text.startswith("{") and text.endswith("}"):
#             data = json.loads(text)
#             return data.get("final_decision", "").strip().lower() == "yes"
        
#         # fallback: try to find first {...} block manually
#         first_brace = text.find("{")
#         last_brace = text.rfind("}")
#         if first_brace != -1 and last_brace != -1:
#             json_chunk = text[first_brace:last_brace+1]
#             data = json.loads(json_chunk)
#             return data.get("final_decision", "").strip().lower() == "yes"

#         raise ValueError("No JSON block found")

#     except Exception as e:
#         print(f"[decision error] Could not parse decision from: {text[:100]} — {e}")
#         return False

def decision(text: str) -> bool:
    """
    Extract the LLM’s yes/no verdict.
    If extraction fails, default to True (‘yes’) to maintain high recall.
    """
    try:
        if not text:
            raise ValueError("empty text")

        # Strip ``` or ```json fences and surrounding whitespace
        text = re.sub(r"^```(?:json)?|```$", "", text.strip(), flags=re.MULTILINE).strip()

        # ---------- 1. strict‑ish JSON ----------
        if text.startswith("{") and text.endswith("}"):
            try:
                data = json.loads(text)
            except json.JSONDecodeError:
                # fix single quotes & trailing commas, then retry
                cleaned = re.sub(r"(')", '"', text)
                cleaned = re.sub(r",\s*([}\]])", r"\1", cleaned)
                data = json.loads(cleaned)

            for k in ("final_decision", "finalDecision", "decision"):
                if k in data:
                    return str(data[k]).strip().lower() == "yes"

        # ---------- 2. regex fallback ----------
        m = re.search(
            r'''(?:final[_\s]?decision|finalDecision|decision)  # key variants
                \s*["']?\s*:\s*["']?\s*
                (?P<ans>yes|no)                                # value
            ''',
            text,
            flags=re.IGNORECASE | re.VERBOSE,
        )
        if m:
            return m.group("ans").lower() == "yes"

        # If we reach here, we didn’t find a usable answer
        raise ValueError("final_decision key not found")

    except Exception as e:
        print(f"[decision warning] Falling back to YES ‑ could not parse: {text[:120]} — {e}")
        return True

### Old implementation

In [ ]:
# # [(article_idx, ent_idx, cand_idx, prompt_str)]
# prompt_records = []          

# for art_idx, output in enumerate(sed_outputs):
#     for ent_idx, ent in enumerate(output["entities"]):
#         ent["candidates"] = retrieve_candidates(ent["entity_title"])
#         for cand_idx, cand in enumerate(ent["candidates"]):
#             prompt_records.append((art_idx, ent_idx, cand_idx, point_wise_el_prompt(ent, cand)))

In [ ]:
# len(prompt_records)

147315

In [ ]:
# with open(f"prompts/prompt_records_4_18_v3.json", "w", encoding="utf-8") as f:
#         json.dump(prompt_records, f, indent=2)
# print('Saved')

Saved


In [ ]:
# for chunk_start in range(0, len(prompt_records), BATCH):
#     chunk = prompt_records[chunk_start: chunk_start+BATCH]
#     prompts = [p[-1] for p in chunk]
#     responses = llm.generate(prompts=prompts, sampling_params=sampling_params)

#     timestamp = datetime.datetime.now().isoformat()
#     batch_out = []

#     for rec, resp in zip(chunk, responses):
#         art_idx, ent_idx, cand_idx, _ = rec
#         txt = resp.outputs[0].text
#         keep = decision(txt)

#         sed_outputs[art_idx]["entities"][ent_idx]["candidates"][cand_idx]["llm_decision"] = txt
#         sed_outputs[art_idx]["entities"][ent_idx]["candidates"][cand_idx]["relevant"] = keep

#         batch_out.append({
#             "article_idx": art_idx,
#             "entity_idx":  ent_idx,
#             "cand_idx":    cand_idx,
#             "keep":        keep,
#             "llm_text":    txt,
#             "timestamp":   timestamp
#         })

#     with open(f"outputs/pointwise/pointwise_filtered/batch_{chunk_start//BATCH+1}_{timestamp.replace(':','-')}.json", "w", encoding="utf-8") as f:
#         json.dump(batch_out, f, indent=2)
#     print(f'Saved batch {chunk_start//BATCH+1}')

In [ ]:
# with open(f"outputs/pointwise/pointwise_filtered/pointwise_sed_outputs_pre_pruning.json", "w", encoding="utf-8") as f:
#     json.dump(sed_outputs, f, indent=2)

In [17]:
for output in sed_outputs:
    for ent in output["entities"]:
        ent["candidates"] = [c for c in ent["candidates"] if c.get("relevant")]

In [19]:
with open(f"outputs/pointwise/pointwise_filtered/pointwise_sed_outputs.json", "w", encoding="utf-8") as f:
    json.dump(sed_outputs, f, indent=2)

In [21]:
chunk_7500 = prompt_records[chunk_start: chunk_start+BATCH]
prompts_7500 = [p[-1] for p in chunk]

In [24]:
chunk_7500[-1], prompts_7500[-1]

((272,
  6,
  4,
  '\n        <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n\n        1. Context: Look at the surrounding text to understand the topic.\n\n        2. Categories: Consider the type of the entity (person, organization, location, etc.).\n\n        3. Modifiers: Pay attention to words or phrases that add details to the mention.\n\n        4. Co-references: Check other mentions of the same entity in the text.\n\n        5. Temporal and Geographical Factors: Consider when and where the text was written.\n\n        6. External Knowledge: Use knowledge from outside the text.\n\n        Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n \n        <|eot_id|><|start_header_id|>user<|end_header_id|>\n\n        Now, I

### Dataframe based inference

In [20]:
skipped_rows = []
wn_val_subset['candidates'] = None
wn_val_subset['pre_pt_len_candidates'] = 0

# [(row_idx, cand_idx, prompt_str)]
for idx, row in wn_val_subset.iterrows():
    row_candidates = retrieve_candidates(row['entity_title'], alias_kb)
    if not row_candidates:
        skipped_rows.append((idx, row['entity_title']))
        continue
        
    wn_val_subset.at[idx, 'candidates'] = row_candidates
    wn_val_subset.at[idx, 'pre_pt_len_candidates'] = len(row_candidates)

In [21]:
# wn_val_subset.to_csv('outputs/pointwise/wn_val_subset_5_7_pre_pt.csv', index=False)

In [76]:
wn_val_subset = pd.read_csv('outputs/pointwise/wn_val_subset_5_7_pre_pt.csv', dtype={'gt_wiki_id': 'Int64'})

In [40]:
# wn_val_subset[wn_val_subset['gt_wiki_id'].isna()==True]
# wn_val_subset = wn_val_subset.dropna(subset=['gt_wiki_id'])

In [77]:
wn_val_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   article_text           250 non-null    object
 1   date                   250 non-null    object
 2   article_title          250 non-null    object
 3   entity_title           250 non-null    object
 4   entity_salience        250 non-null    int64 
 5   offsets                250 non-null    object
 6   wiki_ID                250 non-null    int64 
 7   surrounding_context    250 non-null    object
 8   candidates             237 non-null    object
 9   pre_pt_len_candidates  250 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 19.7+ KB


In [86]:
wn_val_subset['candidates'] = wn_val_subset['candidates'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [87]:
prompt_records = []

for idx, row in wn_val_subset.iterrows():
    row_candidates = row['candidates']
    if not isinstance(row_candidates, list) or not row_candidates:
        continue
        
    for cand_idx, cand in enumerate(row_candidates):
        prompt = construct_pointwise_prompt(row, cand)
        # prompt = pointwise_el_prompt(row, cand)
        prompt_records.append({
            "row_idx": idx,
            "cand_idx": cand_idx,
            "messages": prompt
        })

In [88]:
len(prompt_records)

13984

In [89]:
prompt_records[0]

{'row_idx': 0,
 'cand_idx': 0,
 'messages': [{'role': 'user',
   'content': 'You are an entity disambiguator. I\'ll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text. 5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text. Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Your job is to decide whether a candidate should stay on the shortlist for an entity in context. Output **"yes"** if the candidate is a **plausible match** for the entity mention 

In [90]:
prompt_df = pd.DataFrame(prompt_records)

In [91]:
prompt_df.head()

,row_idx,cand_idx,messages
0,0,0,"[{'role': 'user', 'content': 'You are an entit..."
1,0,1,"[{'role': 'user', 'content': 'You are an entit..."
2,0,2,"[{'role': 'user', 'content': 'You are an entit..."
3,0,3,"[{'role': 'user', 'content': 'You are an entit..."
4,0,4,"[{'role': 'user', 'content': 'You are an entit..."


In [93]:
BATCH = 7500
all_outputs = []

for chunk_start in range(0, len(prompt_df), BATCH):
    chunk = prompt_df.iloc[chunk_start:chunk_start + BATCH]
    messages_batch = list(chunk['messages'])
    # responses = llm.generate(prompts=messages_batch, sampling_params=sampling_params)
    responses = llm.chat(messages=messages_batch, sampling_params=sampling_params)

    outputs = []
    for (i, resp) in enumerate(responses):
        row = chunk.iloc[i]
        txt = resp.outputs[0].text
        keep = decision(txt)

        outputs.append({
            "row_idx": row['row_idx'],
            "cand_idx": row['cand_idx'],
            "keep": keep,
            "llm_text": txt
        })
    all_outputs.extend(outputs)

    print(f"Saved batch {chunk_start//BATCH + 1}")

Processed prompts: 100%|██████████| 7500/7500 [17:05<00:00,  7.31it/s, est. speed input: 5830.93 toks/s, output: 384.39 toks/s]


Saved batch 1


Processed prompts: 100%|██████████| 6484/6484 [15:53<00:00,  6.80it/s, est. speed input: 5868.48 toks/s, output: 362.47 toks/s]


Saved batch 2


In [94]:
import copy

wn_val_subset['candidates_after_pointwise'] = wn_val_subset['candidates'].apply(
    lambda x: copy.deepcopy(x) if isinstance(x, list) else []
)

In [95]:
output_df = pd.DataFrame(all_outputs)

for _, row in output_df.iterrows():
    row_idx = row['row_idx']
    cand_idx = row['cand_idx']
    
    wn_val_subset.at[row_idx, 'candidates_after_pointwise'][cand_idx]['llm_decision'] = row['llm_text']
    wn_val_subset.at[row_idx, 'candidates_after_pointwise'][cand_idx]['relevant'] = row['keep']

In [96]:
wn_val_subset.to_csv('outputs/pointwise/wn_val_subset_5_7_pointwise_pre_pruning.csv', index=False)

In [78]:
# pre_pruning_df = pd.read_csv('wn_val_subset_5_1_pointwise_pre_pruning.csv')

### Prune non‑relevant candidates (in‑place)

In [97]:
wn_val_subset['candidates_after_pointwise'] = wn_val_subset['candidates_after_pointwise'].apply(
    lambda cand_list: [c for c in cand_list if c.get("relevant") is True] if isinstance(cand_list, list) else cand_list
)

In [98]:
wn_val_subset['post_pt_len_candidates'] = wn_val_subset['candidates_after_pointwise'].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

In [99]:
wn_val_subset.to_csv('outputs/pointwise/wn_val_subset_5_7_pointwise_results.csv', index=False)

### Evaluation

In [100]:
wn_val_subset.iloc[0].candidates

[{'mentions': 'australian',
  'wiki_id': 4689264,
  'title': 'Australia',
  'text': "Australia, officially the Commonwealth of Australia, is a sovereign country comprising the mainland of the Australian continent, the island of Tasmania, and numerous smaller islands. It is the largest country in Oceania and the world's sixth-largest country by total area. The neighbouring countries are Papua New Guinea, Indonesia, and East Timor to the north; the Solomon Islands and Vanuatu to the north-east; and New Zealand to the south-east. The population of million is highly urbanised and heavily concentrated on the eastern seaboard. Australia's capital is Canberra, and its largest city is Sydney. The country's other major metropolitan areas are Melbourne, Brisbane, Perth, and Adelaide. Indigenous Australians inhabited the continent for about 65,000 years prior to European discovery with the arrival of Dutch explorers in the early 17th century, who named it New Holland. In 1770, Australia's eastern

In [101]:
wn_val_subset.iloc[0].wiki_ID

np.int64(4689264)

In [102]:
def safe_parse_candidates(x):
    if pd.isna(x):
        return []
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return ast.literal_eval(x)

def compute_metrics_from_pointwise_csv(csv_path):
    df = pd.read_csv(csv_path, dtype={'wiki_ID': 'Int64'})
    df['candidates_after_pointwise'] = df['candidates_after_pointwise'].apply(safe_parse_candidates)

    total_mentions = 0  # Count only mentions that retrieved candidates
    recall_hits = 0
    reduction_ratios = []
    retained_counts = []

    for idx, row in df.iterrows():
        candidates = row['candidates_after_pointwise']
        if not candidates or pd.isna(row['wiki_ID']):
            continue

        total_mentions += 1

        original_count = row['pre_pt_len_candidates']
        retained_count = row['post_pt_len_candidates']
        retained = [c for c in candidates if c.get('relevant') is True]
        
        if original_count == 0:
            continue

        try:
            gt_wiki_id = int(row['wiki_ID'])
        except (ValueError, TypeError):
            continue

        # survived = any(int(c.get('wiki_id')) == int(row['gt_wiki_id']) and c.get('relevant') for c in candidates)
        survived = any(
            c.get('relevant') and 'wiki_id' in c and str(c['wiki_id']).isdigit() and int(c['wiki_id']) == gt_wiki_id
            for c in candidates
        )

        recall_hits += int(survived)
        reduction_ratios.append(retained_count / original_count)
        retained_counts.append(retained_count)

    recall_at_filter = recall_hits / total_mentions if total_mentions > 0 else 0.0
    reduction_ratio = np.mean(reduction_ratios) if reduction_ratios else 1.0
    avg_retained_candidates = np.mean(retained_counts) if retained_counts else 0.0

    return {
        "Recall@filter": recall_at_filter,
        "Reduction ratio": float(reduction_ratio),
        "Average retained candidates": float(avg_retained_candidates)
    }

In [103]:
metrics = compute_metrics_from_pointwise_csv('outputs/pointwise/wn_val_subset_5_7_pointwise_results.csv')
metrics

{'Recall@filter': 0.7838983050847458,
 'Reduction ratio': 0.7268797948905785,
 'Average retained candidates': 28.64406779661017}

In [104]:
with open('outputs/pointwise/wn_val_subset_5_7_pointwise_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

In [106]:
csv_path = 'outputs/pointwise/wn_val_subset_5_7_pointwise_results.csv'

df = pd.read_csv(csv_path, dtype={'wiki_ID': 'Int64'})
df['candidates_after_pointwise'] = df['candidates_after_pointwise'].apply(safe_parse_candidates)

df['retained_count'] = df['candidates_after_pointwise'].apply(
    lambda candidates: sum(1 for c in candidates if c.get('relevant') is True)
)

# Filter only rows with retained_count > 80
outliers_df = df[df['retained_count'] > 50].copy()

# Compute retention ratio and extract retained titles only for outliers
outliers_df['retention_ratio'] = outliers_df['retained_count'] / outliers_df['pre_pt_len_candidates']
outliers_df['retained_titles'] = outliers_df['candidates_after_pointwise'].apply(
    lambda candidates: [c.get('title') for c in candidates if c.get('relevant') is True]
)

outliers_df['entity_title'].unique()

array(['Japan', 'Brazil', 'Mexico', 'Great Britain', 'Olympic Games',
       'Internet', 'Paris', 'Brussels', 'High Court', 'Colombia',
       'football', 'Olympics', 'gymnastics', 'New Zealand', 'San Antonio',
       'Mayor', 'Democratic', '2012 Summer Olympics', 'Green Party',
       'governor', 'President', 'Prime Minister', 'Iran',
       'South Carolina', 'New Mexico', 'Illinois', 'Ohio',
       'Secretary of State', 'campaign', 'Supreme Court', '2008',
       'Republican', 'California'], dtype=object)

## Contextual EL

In [107]:
sampling_params = get_sampling_params(max_tokens=200, temperature=0.6, top_p=0.9, stops=["</s>", "\n}"])

In [108]:
wn_val_subset = pd.read_csv('outputs/pointwise/wn_val_subset_5_7_pointwise_results.csv', dtype={'wiki_ID': 'Int64'})
wn_val_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   article_text                250 non-null    object
 1   date                        250 non-null    object
 2   article_title               250 non-null    object
 3   entity_title                250 non-null    object
 4   entity_salience             250 non-null    int64 
 5   offsets                     250 non-null    object
 6   wiki_ID                     250 non-null    Int64 
 7   surrounding_context         250 non-null    object
 8   candidates                  237 non-null    object
 9   pre_pt_len_candidates       250 non-null    int64 
 10  candidates_after_pointwise  250 non-null    object
 11  post_pt_len_candidates      250 non-null    int64 
dtypes: Int64(1), int64(3), object(8)
memory usage: 23.8+ KB


In [109]:
import ast

wn_val_subset['candidates_after_pointwise'] = wn_val_subset['candidates_after_pointwise'].apply(
    lambda x: ast.literal_eval(x) if pd.notna(x) else []
)

In [110]:
COT_POOL = [
    {
        "mention": "Washington, D.C.",
        "surrounding_context": "The United States capital of ###Washington, D.C.### legalized same-sex marriage on Wednesday.",
        "candidates": [
            {"id": 1, "name": "Washington, D.C.", "summary": "The capital city of the United States."},
            {"id": 2, "name": "Washington (state)", "summary": "A state in the Pacific Northwest region of the United States."}
        ],
        "answer": "1.Washington, D.C."
    },
    {
        "mention": "Eric Massa",
        "surrounding_context": "New York Representative ###Eric Massa### announced Wednesday that he would be stepping down as Congressman from New York's 29th congressional district.",
        "candidates": [
            {"id": 1, "name": "Eric Massa", "summary": "A former U.S. Congressman from New York."},
            {"id": 2, "name": "Eric Massan", "summary": "A fictional character in a novel."}
        ],
        "answer": "1.Eric Massa"
    },
    {
        "mention": "Air Canada",
        "surrounding_context": "Canadian airline ###Air Canada### has said that it will be laying off 1,010 machinists seconded to work at the Aveos Fleet Performance aircraft maintenance company.",
        "candidates": [
            {"id": 1, "name": "Air Canada", "summary": "The flag carrier and largest airline of Canada."},
            {"id": 2, "name": "Canada Air", "summary": "A fictional airline in a movie."}
        ],
        "answer": "1.Air Canada"
    },
    {
        "mention": "Bosnian president",
        "surrounding_context": "Former ###Bosnian president### Dr. Ejup Ganić was arrested by the Metropolitan Police at Heathrow Airport, London on Monday.",
        "candidates": [
            {"id": 1, "name": "Ejup Ganić", "summary": "A former president of the Federation of Bosnia and Herzegovina."},
            {"id": 2, "name": "Alija Izetbegović", "summary": "The first president of Bosnia and Herzegovina."}
        ],
        "answer": "1.Ejup Ganić"
    },
    {
        "mention": "Greek bailout",
        "surrounding_context": "The spike lower this morning indicates market nervousness about the prospects of a ###Greek bailout### - the message coming out of Europe is still confused.",
        "candidates": [
            {"id": 1, "name": "Greek bailout", "summary": "A financial assistance program for Greece during its debt crisis."},
            {"id": 2, "name": "Greek mythology", "summary": "A body of myths originally told by the ancient Greeks."}
        ],
        "answer": "1.Greek bailout"
    }
]

In [48]:
import random
import textwrap

def contextual_el_chat(entity_row):
    ex = random.choice(COT_POOL)

    ex_candidates = "; ".join([f"{c['id']}.{c['name']} – {c['summary']}" for c in ex['candidates']])

    return [
        {
            "role": "user",
            "content": (f"""You are an entity disambiguator. I'll provide you with a detailed description of entity disambiguation along with several tips on what to look out for:\n 1. Context: Look at the surrounding text to understand the topic. 2. Categories: Consider the type of the entity (person, organization, location, etc.). 3. Modifiers: Pay attention to words or phrases that add details to the mention. 4. Co-references: Check other mentions of the same entity in the text.  5. Temporal and Geographical Factors: Consider when and where the text was written. 6. External Knowledge: Use knowledge from outside the text.  Remember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment. Now, I will first provide you with an example to illustrate the task: Mention: {ex['mention']} Context: {ex['surrounding_context']} Candidates: {ex_candidates} Answer: {ex['answer']}. Now, I will give you a new mention, its context, and a list of candidate entities. The mention is highlighted with '###'. Mention: {entity_row['entity_title']} Context: {entity_row['surrounding_context']} Candidates: """ + "; ".join([f"{idx + 1}.{cand['title']} – {cand['text'][:300]}"  for idx, cand in enumerate(entity_row['candidates_after_pointwise'])]) + """ Think step by step. At the end output exactly one line with the ID and name of the chosen entity, e.g., '3.Barack Obama'. If none of the candidates are correct, output '-1.None'. Only output a single JSON object in the following format, without line breaks, indentation, or extra text: {"final_decision": "<id>.<entity name>", "reasoning": "your explanation here"} """)
        }
    ]

In [111]:
c_prompt = contextual_el_chat(wn_val_subset.iloc[0])

In [112]:
op = llm.chat(messages=c_prompt, sampling_params=sampling_params)
op[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.59s/it, est. speed input: 333.28 toks/s, output: 32.35 toks/s]


'{"final_decision": "1.Australia", "reasoning": "The mention \'Australian\' in the context of the text \'Homebush Bay, New South Wales —Earlier today, the ###Australian### Gliders beat the Chinese women\'s national wheelchair basketball team\' suggests that the entity being referred to is the country of Australia, given that \'Gliders\' is the nickname of the Australian women\'s national wheelchair basketball team. This is also supported by the fact that the game is taking place at the Sydney Olympic Park, which is a well-known location in Australia."}'

In [113]:
parse_contextual_el_output(op[0].outputs[0].text)

1

In [51]:
def parse_contextual_el_output(text):
    try:
        match = re.search(r'\{.*?\}', text, re.DOTALL)
        if not match:
            return -1
        
        data = json.loads(match.group())
        decision = data.get("final_decision", "").strip()
        
        # Expecting something like "3.Tesla, Inc."
        m = re.match(r'(-?\d+)\s*\.\s*(.+)', decision)
        if not m:
            return -1
        
        label = int(m.group(1))
        return label

    except Exception as e:
        print(f"[parse_contextual_el_output] Parsing error: {e}")
        return -1

In [114]:
prompt_cel_records = []

for idx, row in wn_val_subset.iterrows():
    if not row['candidates_after_pointwise']:
        continue
    prompt = contextual_el_chat(row)
    label_map = {i+1: cand for i, cand in enumerate(row['candidates_after_pointwise'])}
    prompt_cel_records.append((idx, label_map, prompt))

In [115]:
BATCH_SIZE = 7500

all_outputs = []

for chunk_start in range(0, len(prompt_cel_records), BATCH_SIZE):
    batch = prompt_cel_records[chunk_start:chunk_start + BATCH_SIZE]
    prompts = [rec[2] for rec in batch]

    responses = llm.chat(messages=prompts, sampling_params=sampling_params)

    for (record, response) in zip(batch, responses):
        idx, label_map, _ = record
        text = response.outputs[0].text.strip()
        selected_label = parse_contextual_el_output(text)

        if selected_label > 0 and selected_label in label_map:
            selected_candidate = label_map[selected_label]
        else:
            selected_candidate = 0

        all_outputs.append((idx, selected_candidate))

Processed prompts: 100%|██████████| 236/236 [02:47<00:00,  1.41it/s, est. speed input: 3584.06 toks/s, output: 174.13 toks/s]

[parse_contextual_el_output] Parsing error: Expecting ',' delimiter: line 1 column 67 (char 66)


In [116]:
# top_linked_entities = [None] * len(wn_val_subset)

# for idx, selected_candidate in all_outputs:
#     if selected_candidate:
#         top_linked_entities[idx] = selected_candidate['wiki_id']
#     else:
#         top_linked_entities[idx] = 0

# wn_val_subset['top_linked_entity'] = top_linked_entities

top_linked_entities = [0] * len(wn_val_subset)

for idx, selected_candidate in all_outputs:
    if selected_candidate:
        top_linked_entities[idx] = int(selected_candidate['wiki_id'])

wn_val_subset['top_linked_entity'] = pd.Series(top_linked_entities, dtype="Int64")

In [117]:
wn_val_subset.to_csv('outputs/contextual/wn_val_subset_5_7_contextual_results_v2.csv', index=False)

In [118]:
def evaluate_contextual_linking(df):
    """
    Evaluate performance of contextual entity linker.
    Assumes 'top_linked_entity' and 'wiki_ID' are both integers, with 0 meaning "no prediction made".
    """

    total_mentions = len(df)
    linked_mentions = (df['top_linked_entity'] != 0).sum()
    ground_truth_mentions = df['wiki_ID'].notna().sum()

    correct_links = 0

    for _, row in df.iterrows():
        pred = row['top_linked_entity']
        gt = row['wiki_ID']

        if pred == 0 or pd.isna(gt):
            continue

        if int(pred) == int(gt):
            correct_links += 1

    accuracy = correct_links / total_mentions if total_mentions else 0
    precision_at_linked = correct_links / linked_mentions if linked_mentions else 0
    recall = correct_links / ground_truth_mentions if ground_truth_mentions else 0
    
    if precision_at_linked + recall > 0:
        f1_score = 2 * (precision_at_linked * recall) / (precision_at_linked + recall)
    else:
        f1_score = 0.0

    print(f"Entities evaluated: {total_mentions}")
    print(f"Ground truth (non-null): {ground_truth_mentions}")
    print(f"Predictions made: {linked_mentions}")
    print(f"Correct links: {correct_links}\n")

    return {
        "Accuracy": accuracy,
        "Precision@Linked": float(precision_at_linked),
        "Recall": float(recall),
        "F1 Score": float(f1_score)
    }


In [119]:
contextual_metrics = evaluate_contextual_linking(wn_val_subset)
contextual_metrics

Entities evaluated: 250
Ground truth (non-null): 250
Predictions made: 205
Correct links: 145



{'Accuracy': 0.58,
 'Precision@Linked': 0.7073170731707317,
 'Recall': 0.58,
 'F1 Score': 0.6373626373626374}

In [120]:
with open('outputs/contextual/wn_val_subset_5_7_contextual_metrics.json', 'w') as f:
    json.dump(contextual_metrics, f, indent=4)